<a href="https://colab.research.google.com/github/Parshav-Shah/ISYS5002_portfolio/blob/main/Key_Execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ASSIGNMENT**



1. This notebook will prompt the user to input a company's stock market ticker symbol. 
2. It will then get the data from the Yahoo Finance website to extract information on the key of the company. 
3. Then the income tax will be calculated for each
executive based on the ATO tax tables. 
4. The User can further choose if the data
will be saved as a CSV and/or stored in an SQLite database.








 # **Algorithm**

*1. First Of all, User will enter the name of Ticker.*

*2. Ticker symbol will be used to extract Web page and scrape data of Key Executives of that Stock ticker.*

*3. A function named get_url is created to create connection with mentioned URL.*

*4. Then a function will be created to extract cells data of each row.*

*5. Finally, The function will be created to get salary of all key executives and calculate tax on the pay based on the ATO tables.* 


# Install Unique Packages:-


In [ ]:
# Install unique packages
!pip install kora -q

# load packages
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd
import csv


# Create a function to create connection with the URL:-





In [ ]:
def get_url():
  '''This function will prompt the user to input a ticker symbol and get 
  the URL of the profile page'''
  
  ticker_symbol = input( "Please input a ticker symbol ")
  print(ticker_symbol)
  example_url = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  result = example_url.format(ticker_symbol, ticker_symbol)
  return result

# Function to get the Cells data of the generated table:-



In [ ]:
def get_cells_data(row):
  ''' This function will get the data of the first cells(Name, Title, Pay 
  of the Key Executives''' 
  
  cells = row.find_all('td')
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  return {'name': name, 'title': title,'pay': pay}
  

# function to extract records of Key Executives:-


In [ ]:
def get_extract_records(soup):
  '''This function will extract records of key executives'''
 
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  
  extract_records = []
  for row in rows:
    result = get_cells_data(row)
    extract_records.append(result)
  return extract_records
  

# Function to get the salary of the Key Executives:-


  


In [ ]:
def get_salary(rec):
  ''' This function will get the salary(pay) of the Key executives'''
  
  pay = rec['pay']
  last_pay_character = pay[-1]
  num_part = pay[0:-1]

  if last_pay_character == 'M':
    salary = float(num_part) * 1000000
    #print("Millions")
  elif last_pay_character == 'k':
    salary = float(num_part) * 1000
    #print("Thousand")
  else:
    salary = 0
    #print("Not sure")

  return salary
  

# Function to Calculate the Income tax(based on ATO table) on salary of the Key Executives:-




In [ ]:
def calculate_tax(income):
   ''' This function will calculate the tax (based on ATO table) 
   on salary(pay) of the Key executives'''
   
   if income <= 18200:
    tax_payable = 0
   elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
   elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
   elif 120001 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
   elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)
    
    return tax_payable
   

#Final step is to scrape the Yahoo Finance Table of Key Executives :-

In [ ]:
# Get URL

''' This will help to scrape the table of the stock ticker symbol input 
by the user and include tax column as well'''

url = get_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
records = get_extract_records(soup)

for record in records:
  result = get_salary(record)
  income_tax = calculate_tax(result)
  record['tax'] = income_tax

# Create a pandas dataframe 

execs_df = pd.DataFrame.from_records(records)
execs_df

Please input a ticker symbol NAB.Ax
NAB.Ax


,name,title,pay,tax
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2.54M,1113667.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1.13M,479167.0
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1.72M,744667.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912.98k,381508.0
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929.27k,388838.5
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1.62M,699667.0
6,Mr. Shaun Dooley,Chief Risk Officer,1.05M,443167.0
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333.27k,120638.5
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,N/A,NaN
9,Ms. Sally Mihell,Head of Investor Relations,N/A,NaN


# Saving the Output Results in User Selected Format:-






In [ ]:
A = "CSV Format"
B = "Database"
C = "CSV and Database"
print("A : CSV Format") 
print("B : DataBase")
print("C : In both Format")
D = input("Select above option A,B or C to store Data??")
if D == 'A':
  '''Saving the Income Tax of Key Executives in a csv and an excel file'''
  # Saving the output
  df.to_csv('Key_Executives.csv')

  print('Saved in CSV File Format....')
elif D == 'B':
  #Saving the Income tax of Key Executives to SQLite Database using pandas dataframe..
  import sqlite3
  conn = sqlite3.connect("Key_Executives.db")
  execs_df.to_sql('execs_df',conn,if_exists='append', index=False)
  Name= pd.read_sql('SELECT * FROM execs_df', conn)
  Name.head()
  print("Database Saved in Files Folder of Google Colab...")
elif D == 'C':
  '''Saving the Income Tax of Key Executives in a csv and an excel file'''
  # Saving the output
  execs_df.to_csv('Key_Executives.csv')

  print('Saved in CSV File Format....')
  import sqlite3
  conn = sqlite3.connect("Key_Executives.db")
  execs_df.to_sql('execs_df',conn,if_exists='append', index=False)
  Name= pd.read_sql('SELECT * FROM execs_dff', conn)
  Name.head()
  print("Database Saved in Files Folder of Google Colab...")
else:
  print("Select Correct option...")

A : CSV Format
B : DataBase
C : In both Format
Select above option A,B or C to store Data??A
Saved in CSV File Format....
